In [1]:
# Third data compilation file, takes each protein's sequence and queries the prosite API for unique motif signatures
# Adds each unique motif as a column in the initialSet csv file, one hot encoded with each motif being initially set to 0
# Adds a Processed_Motifs column at the end for easier processing in the next step

# ONLY NEEDS TO BE RUN ONCE

import pandas as pd
import requests
import numpy as np
from tqdm import tqdm

# ✅ File path (customize if needed)
input_file = "/Users/carmenshero/Desktop/Datasets/initialSet.csv"

# ✅ Load dataset (must contain PDB_ID, UniProt_ID, and sequence)
df = pd.read_csv(input_file, dtype={"sequence": str}, low_memory=False)

# ✅ Clean up sequences
df["sequence"] = df["sequence"].replace(["nan", np.nan], "")
df["sequence"] = df["sequence"].fillna("").astype(str)

# ✅ Function to query motifs from PROSITE
def get_protein_motifs(sequence):
    """Return a list of PROSITE motifs for a given protein sequence."""
    if sequence.strip() == "" or sequence == "N/A":
        return []

    url = "https://prosite.expasy.org/cgi-bin/prosite/PSScan.cgi"
    params = {"seq": sequence, "output": "json"}
    response = requests.get(url, params=params)

    if response.status_code != 200:
        return []

    data = response.json()
    return [match.get("signature_ac") for match in data.get("matchset", [])]

# ✅ Step 1: Find all unique motifs across the dataset
unique_motifs = set()

for sequence in tqdm(df["sequence"].unique(), desc="Discovering motifs"):
    if sequence.strip() and sequence != "N/A":
        motifs = get_protein_motifs(sequence)
        unique_motifs.update(motifs)

print(f"✅ Found {len(unique_motifs)} unique motifs.")

# ✅ Step 2: Add motif columns, initialize to 0
for motif in sorted(unique_motifs):
    if motif not in df.columns:
        df[motif] = 0

# ✅ Step 3: Add Processed_Motifs column
if "Processed_Motifs" not in df.columns:
    df["Processed_Motifs"] = False

# ✅ Save updated dataset
df.to_csv(input_file, index=False)
print(f"💾 Saved updated dataset with motif columns to {input_file}")


Discovering motifs: 100%|██████████████████████████████████████████████████████████████████████| 75/75 [03:41<00:00,  2.95s/it]

✅ Found 95 unique motifs.
💾 Saved updated dataset with motif columns to /Users/carmenshero/Desktop/Datasets/initialSet.csv


In [2]:
from tqdm import tqdm
import time

# ✅ Load updated dataset with motif columns and Processed_Motifs flag
input_file = "/Users/carmenshero/Desktop/Datasets/initialSet.csv"
df = pd.read_csv(input_file, dtype={"sequence": str}, low_memory=False)

# ✅ Ensure sequences are cleaned
df["sequence"] = df["sequence"].fillna("").astype(str)

# ✅ PROSITE query function (in case not already defined)
def get_protein_motifs(sequence):
    if sequence.strip() == "" or sequence == "N/A":
        return []

    url = "https://prosite.expasy.org/cgi-bin/prosite/PSScan.cgi"
    params = {"seq": sequence, "output": "json"}
    response = requests.get(url, params=params)

    if response.status_code != 200:
        return []

    data = response.json()
    return [match.get("signature_ac") for match in data.get("matchset", [])]

# ✅ Set batch size
batch_size = 100
processed_count = 0

# ✅ Iterate in batches
for start_idx in tqdm(range(0, len(df), batch_size), desc="Processing motifs in batches"):
    end_idx = min(start_idx + batch_size, len(df))
    batch = df.iloc[start_idx:end_idx]

    updated = False

    for i in range(len(batch)):
        index = start_idx + i
        row = df.loc[index]
        sequence = row["sequence"]

        if row["Processed_Motifs"] or not sequence or sequence.strip() == "" or sequence == "N/A":
            continue

        try:
            motifs = get_protein_motifs(sequence)

            if motifs:
                print(f"✔ Motifs for index {index}: {motifs}")
                for motif in motifs:
                    if motif in df.columns:
                        df.at[index, motif] = 1
                    else:
                        print(f"⚠️ Motif '{motif}' not in columns. Skipped.")
            else:
                print(f"⚠️ No motifs found for index {index}")

            df.at[index, "Processed_Motifs"] = True
            processed_count += 1
            updated = True

        except Exception as e:
            print(f"❌ Error processing index {index}: {e}")
            continue

    # ✅ Save after each batch if updates occurred
    if updated:
        df.to_csv(input_file, index=False)
        print(f"💾 Saved batch {start_idx}-{end_idx}. Total processed so far: {processed_count}")

print("✅ All batches complete.")


Processing motifs in batches:   0%|                                                                      | 0/1 [00:00<?, ?it/s]

✔ Motifs for index 0: ['PS50404', 'PS50405']
⚠️ No motifs found for index 1
⚠️ No motifs found for index 2
✔ Motifs for index 3: ['PS00236']
✔ Motifs for index 4: ['PS50168', 'PS50017']
✔ Motifs for index 5: ['PS00061']
✔ Motifs for index 6: ['PS01159', 'PS01164', 'PS01165']
⚠️ No motifs found for index 7
✔ Motifs for index 8: ['PS01033']
✔ Motifs for index 9: ['PS01033']
✔ Motifs for index 10: ['PS51007']
✔ Motifs for index 11: ['PS50072', 'PS00170']
✔ Motifs for index 12: ['PS00167']
✔ Motifs for index 13: ['PS00168']
✔ Motifs for index 14: ['PS51856']
✔ Motifs for index 15: ['PS50835', 'PS00290']
✔ Motifs for index 16: ['PS50835', 'PS00290']
✔ Motifs for index 17: ['PS51162', 'PS00484']
✔ Motifs for index 18: ['PS01033']
✔ Motifs for index 19: ['PS50835']
✔ Motifs for index 20: ['PS50835']
✔ Motifs for index 21: ['PS50835']
✔ Motifs for index 22: ['PS50835']
✔ Motifs for index 23: ['PS00102']
✔ Motifs for index 24: ['PS00859', 'PS00860']
✔ Motifs for index 25: ['PS00436', 'PS50873',

Processing motifs in batches: 100%|█████████████████████████████████████████████████████████████| 1/1 [04:11<00:00, 251.10s/it]

✔ Motifs for index 84: ['PS60025']
💾 Saved batch 0-85. Total processed so far: 85
✅ All batches complete.
